In [2]:
import pandas as pd
import researchpy as rp
import scipy.stats as stats

In [3]:
df_origin = pd.read_csv('./DILI/final/DILI_Features_cleaned.csv')
df_origin

FileNotFoundError: [Errno 2] No such file or directory: './DILI/final/DILI_Features_cleaned.csv'

In [121]:
df_tar = df_origin[['DILI']]
df_desc = df_origin.loc[:,'MaxEStateIndex':]
df = pd.concat([df_tar,df_desc],axis=1)
df['DILI'].replace({1:'Positive', 0:'Negative'}, inplace=True)
df

,DILI,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Positive,6.193380,0.092361,6.193380,0.092361,0.848995,255.361,234.193,255.162314,100,...,0,0,0,0,0,0,0,0,0,0
1,Negative,6.482112,-0.455333,6.482112,0.455333,0.722715,343.898,317.690,343.170292,128,...,0,0,0,0,0,0,0,0,0,0
2,Positive,6.271931,0.815501,6.271931,0.815501,0.817874,314.860,291.676,314.154976,116,...,0,0,0,0,0,0,0,0,0,0
3,Positive,6.176540,0.706466,6.176540,0.706466,0.800122,326.831,307.679,326.129824,118,...,0,0,0,0,0,0,0,0,0,0
4,Negative,13.195604,-0.817283,13.195604,0.137241,0.622271,426.560,396.320,426.230728,164,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Negative,14.188117,-2.026666,14.188117,0.171241,0.433136,583.689,546.393,583.279469,224,...,0,0,0,0,0,0,0,0,0,0
2222,Negative,9.480602,-0.669907,9.480602,0.180178,0.510157,183.207,170.103,183.089543,72,...,0,0,0,0,0,0,0,0,0,0
2223,Negative,10.324075,-0.664234,10.324075,0.331507,0.290517,282.468,248.196,282.255880,118,...,0,0,0,0,0,0,0,0,10,0
2224,Negative,5.273144,0.205596,5.273144,0.205596,0.513143,198.314,176.138,198.184447,82,...,0,0,0,0,0,0,0,0,0,0


In [122]:
rp.summary_cont(df['fr_urea'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,fr_urea,2226.0,0.0373,0.2161,0.0046,0.0283,0.0463


In [123]:
rp.summary_cont(df['fr_urea'].groupby(df['DILI']))

,N,Mean,SD,SE,95% Conf.,Interval
DILI,,,,,,
Negative,1014,0.0345,0.2031,0.0064,0.0220,0.0470
Positive,1212,0.0396,0.2264,0.0065,0.0268,0.0524


In [124]:
#[0]: F-statistics / [1]: p-value
stats.f_oneway(df['fr_urea'][df['DILI'] == 'Positive'],
               df['fr_urea'][df['DILI'] == 'Negative'])[1]

0.5802845016109147

In [125]:
stats.f_oneway(df.iloc[:,1][df['DILI'] == 'Positive'],
               df.iloc[:,1][df['DILI'] == 'Negative'])[1]

0.9132263737532695

In [126]:
df.iloc[:,208]

0       0
1       0
2       0
3       0
4       0
       ..
2221    0
2222    0
2223    0
2224    0
2225    0
Name: fr_urea, Length: 2226, dtype: int64

In [130]:
pdict = dict()
for i in range(208):
    name = df.iloc[:,i+1].name
    try:
        temp = stats.f_oneway(df.iloc[:,i+1][df['DILI'] == 'Positive'],
                              df.iloc[:,i+1][df['DILI'] == 'Negative'])[1]
        pdict[name] = temp
    except Exception as e:
        print(e)
        pdict[name] = None

In [131]:
pval = pd.DataFrame.from_dict(pdict,orient='index')
pval.to_csv('./DILI/final/p-value.csv')

In [132]:
sig_pdict = dict()
n=1
for i in sorted(pdict.items(), key=lambda item:item[1]):
    if i[1] < 0.05:
        sig_pdict[i[0]]=i[1]
        n +=1

In [133]:
sig_pval = pd.DataFrame.from_dict(sig_pdict,orient='index')
sig_pval.sort_values(by=0)
sig_pval

,0
BCUT2D_LOGPLOW,1.727633e-12
PEOE_VSA1,5.951604e-10
BCUT2D_MRLOW,1.039506e-09
Chi4v,6.643021e-08
Chi3v,1.518574e-07
...,...
fr_phenol_noOrthoHbond,3.395824e-02
fr_epoxide,3.558434e-02
fr_halogen,4.205273e-02
fr_aryl_methyl,4.577646e-02


In [135]:
sig_pval.to_csv('./DILI/final/p-value_significant.csv')